# 0. selenium 설치

In [ ]:
!pip install chromedriver_autoinstaller

In [ ]:
!pip install selenium

# 1. 데이터 불러오기

In [88]:
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함


from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm


# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time


# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 크롤링

In [89]:
def open_chrome():
    ## 크롬 윈도우 설정
    # 크롬 드라이버 위치 설정
    chrome_path = chromedriver_autoinstaller.install()


    # 크롬 옵션
    options = webdriver.ChromeOptions()
    # 크롬 윈도우 사이즈 조절
    options.add_argument("--window-size=800,1400")

    driver = webdriver.Chrome(chrome_path, options=options)

    # 실행할 시간 랜덤값 지정
    A = np.random.randint(1,3)
    
    return driver , A

##### 참고


- find_elements_by_tag_name:          Copy Element의 tag
- find_elements_by_xpath:             Copy XPath 또는 Copy full XPath
- find_elements_by_id:                Copy Element의 id attribute
- find_elements_by_class_name:        Copy Element의 class attribute
- find_elements_by_link_text:         Copy Element의 text
- find_elements_by_partial_link_text: Copy Element의 text중 일부
- find_elements_by_css_selector:      Copy selector


- '요소' 클릭:        	.click('')   
- '요소' 더블 클릭:	     .double_click('')   
- '텍스트' 입력:	         .send_keys('')  
- '요소'에 '텍스트' 입력:	  .send_keys_to_element('','')  
- '요소'로 마우스 이동:	  .move_to_element('')
- 스크롤 내리기:	         .execute_script("window.scrollto();")
- 실행:                  .perform()

## 2-1. 팀 컬러 크롤링



- 팀 컬러 명
- 단계
- 적용 효과

*특성 팀 컬러의 경우'적용 선수' 추가

### 2-1-1. 팀 컬러 목록 part

In [6]:
# chrome 창 열기
driver, A = open_chrome()

## 팀 컬러 목록 크롤링
# 팀 컬러 타입 리스트
teamcolor_club = []
teamcolor_nation = []
teamcolor_reinforce = []
teamcolor_relation = []
teamcolor_special =[]

# 팀 컬러 정보 목록 크롤링 파트 함수
def TC_list_crawling():
    driver.find_element_by_class_name('btn_search').click()
    time.sleep(A)

    locates = 'div.name'
    elements = driver.find_elements_by_css_selector(locates)
    
    return elements



# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)
time.sleep(A)


# 팝업창 닫기 (팝업창 오류 시 사용)
driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
time.sleep(A)

# 팀 컬러 정보 목록 저장
for x in tqdm(range(2,7)):
        driver.find_element_by_xpath(f'//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[{x}]/label').click()
        time.sleep(A)
        
        elements = TC_list_crawling()

        ## 클럽
        if x == 2:
                for element in elements:
                        teamcolor_club.append(element.text)
                
        elif x == 3:
                for element in elements:
                        teamcolor_nation.append(element.text)
        
        elif x == 4:
                for element in elements:
                        teamcolor_reinforce.append(element.text)
                        
        elif x == 5:
                for element in elements:
                        teamcolor_relation.append(element.text)
                        
        elif x == 6:
                for element in elements:
                        teamcolor_special.append(element.text)
        
        else:
                print('not in range')
        
        driver.find_element_by_class_name('btn_reset').click()
        time.sleep(A)
        

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
# 팀 컬러 목록 데이터 프레임 저장
TC_club = pd.DataFrame(data = teamcolor_club, columns = ['team color'])
TC_nation = pd.DataFrame(data = teamcolor_nation, columns = ['team color'])
TC_reinforce = pd.DataFrame(data = teamcolor_reinforce, columns = ['team color'])
TC_relation = pd.DataFrame(data = teamcolor_relation, columns = ['team color'])
TC_special = pd.DataFrame(data = teamcolor_special, columns = ['team color'])

# # csv로 파일 저장
# TC_club.to_csv("TC_club.csv", encoding='utf-8-sig', index = False)
# TC_nation.to_csv("TC_nation.csv", encoding='utf-8-sig', index = False)
# TC_reinforce.to_csv("TC_reinforce.csv", encoding='utf-8-sig', index = False)
# TC_relation.to_csv("TC_relation.csv", encoding='utf-8-sig', index = False)
# TC_special.to_csv("TC_special.csv", encoding='utf-8-sig', index = False)


### 2-1-2. 팀 컬러 세부 사항 part

In [3]:
# csv 파일 불러오기
TC_club = pd.read_csv('./data/team_color_list/TC_club.csv')
TC_nation = pd.read_csv('./data/team_color_list/TC_nation.csv')
TC_reinforce = pd.read_csv('./data/team_color_list/TC_reinforce.csv')
TC_relation = pd.read_csv('./data/team_color_list/TC_relation.csv')
TC_special = pd.read_csv('./data/team_color_list/TC_special.csv')

# 다시 리스트 형식으로 변경
teamcolor_club = TC_club['team color']
teamcolor_nation = TC_nation['team color']
teamcolor_reinforce = TC_reinforce['team color']
teamcolor_relation = TC_relation['team color']
teamcolor_special = TC_special['team color']

# 팀 컬러 타입 모음
teamcolor_type = [teamcolor_club,teamcolor_nation,teamcolor_reinforce,teamcolor_relation,teamcolor_special]

In [4]:
# css selector 위치 입력기
def location_inputor(num,type):
    if type == 'div':
        temp_1 = driver.find_element_by_css_selector('div > div.lv_content > div.desc').text
        temp_2 = driver.find_element_by_css_selector('div > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    elif type == 'div.level.lvu':
        temp_1 = driver.find_element_by_css_selector(f'div.level.lvu{num} > div.lv_content > div.desc').text
        temp_2 = driver.find_element_by_css_selector(f'div.level.lvu{num} > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    elif type == 'div.level.lv':    
        temp_1 = driver.find_element_by_css_selector(f'div.level.lv{num} > div.lv_content > div.desc').text
        temp_2 = driver.find_element_by_css_selector(f'div.level.lv{num} > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    return temp_1, temp_2

In [55]:
# chrome 창 열기
driver, A = open_chrome()


## 팀 컬러 타입 별 적용 조건 & 효과 저장
# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)

count = 0

for teamcolors in tqdm(teamcolor_type):
   # 크롤링한 결과를 저장할 리스트
   teamcolor_info = []
   step_1_require_player_nums = []
   step_2_require_player_nums = []
   step_3_require_player_nums = []
   step_4_require_player_nums = []
   step_1_effects = []
   step_2_effects = []
   step_3_effects = []
   step_4_effects = []
   
   for x in tqdm(teamcolors):
      # # 팝업창 닫기
      # driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
      # time.sleep(A)
      try:
         
         # 팀 컬러 타입 선택
         if teamcolors == teamcolor_club:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[2]/label').click()
         elif teamcolors == teamcolor_nation:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[3]/label').click() 
         elif teamcolors == teamcolor_reinforce:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[4]/label').click()
         elif teamcolors == teamcolor_relation:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[5]/label').click() 
         elif teamcolors == teamcolor_special:
            driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[6]/label').click()  
         
         time.sleep(1)
         
         
         # 팀 컬러 입력 & 검색
         driver.find_element_by_class_name('ui-autocomplete-input').click()
         driver.find_element_by_class_name('ui-autocomplete-input').send_keys(x)
         driver.find_element_by_class_name('btn_search').click()
         time.sleep(A)
         
         
         # 단계 추출 (for 문 돌릴 갯수 정하기 용도)
         number = driver.find_element_by_css_selector('div.teamcolor_item_list > div > div.level').text
         number = int(number.replace('단계',''))


         # 세부 설명 들어가기
         driver.find_element_by_class_name('name').click()
         driver.find_element_by_class_name('btn_detail_link').click()
         time.sleep(A) 


         # 팀 컬러 설명 저장
         info = driver.find_element_by_css_selector('#teamcolorPop > div > div.header > div > span').text
         teamcolor_info.append(info)
         time.sleep(1)
         
         
         
         # 단계 별 적용 조건 & 적용 효과 저장
         for num in range(1,5):
            # 1단계
            if num == 1:
               if number == 1:
                  # 'num'과 아래의 타입 중 하나를 골라서 넣으세요!
                  # type = 'div', 'div.level.lvu', 'div.level.lv'
                  temp_1, temp_2 = location_inputor(num,'div')
                  
                  step_1_require_player_nums.append(temp_1)
                  step_1_effects.append(temp_2)
                  
               elif number > 1:   
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
               
                  step_1_require_player_nums.append(temp_1)
                  step_1_effects.append(temp_2)
               else:
                  step_1_require_player_nums.append('-')
                  step_1_effects.append('-') 

            # 2단계
            elif num == 2:
               if num <= number:
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
                  
                  step_2_require_player_nums.append(temp_1)
                  step_2_effects.append(temp_2)
                  
               else:
                  step_2_require_player_nums.append('-')
                  step_2_effects.append('-') 
               
            # 3단계
            elif num == 3:
               if num <= number:
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
                  
                  step_3_require_player_nums.append(temp_1)
                  step_3_effects.append(temp_2)
                  
               else:
                  step_3_require_player_nums.append('-')
                  step_3_effects.append('-') 

            # 4단계
            elif num == 4:
               if num <= number:
                  if teamcolors == teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lvu')
                  
                  elif teamcolors != teamcolor_reinforce:
                     temp_1, temp_2 = location_inputor(num,'div.level.lv')
                  
                  step_4_require_player_nums.append(temp_1)
                  step_4_effects.append(temp_2)
                  
               else:
                  step_4_require_player_nums.append('-')
                  step_4_effects.append('-') 

            else:
               print('error')
               
            time.sleep(1)
            
         # 세부 설명 나오기
         driver.find_element_by_class_name('btn_close').click()
         time.sleep(1)

         # 검색어 초기화 
         driver.find_element_by_class_name('btn_reset').click()
         time.sleep(1)
      
      except:
         print('error: 팀 컬러 선택 & 상세 정보 클릭')
         break
      
   
   
   ## 팀 컬러 하나 완료하면 csv로 저장            
   # dict로 변형 & 데이터 프레임화
   dict_temp = {  '팀 컬러': [val for val in teamcolors],
                  '팀 컬러 설명': [val for val in teamcolor_info],             
                  '1단계 적용 조건': [val for val in step_1_require_player_nums],
                  '2단계 적용 조건': [val for val in step_2_require_player_nums],
                  '3단계 적용 조건': [val for val in step_3_require_player_nums],
                  '4단계 적용 조건': [val for val in step_4_require_player_nums],
                  '1단계 효과': [val for val in step_1_effects], 
                  '2단계 효과': [val for val in step_2_effects],
                  '3단계 효과': [val for val in step_3_effects],
                  '4단계 효과': [val for val in step_4_effects]
   }
   columns = ['팀 컬러','팀 컬러 설명','1단계 적용 조건','2단계 적용 조건','3단계 적용 조건','4단계 적용 조건','1단계 효과','2단계 효과','3단계 효과','4단계 효과']
   
   teamcolor_naming = ['클럽_팀_컬러','국가_팀_컬러','강화_팀_컬러','관계_팀_컬러','스페셜_팀_컬러']

   globals()[f'df_{teamcolor_naming[count]}'] = pd.DataFrame(data = dict_temp, columns=columns)
   count += 1
   
   time.sleep(A)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/206 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [59]:
## 데이터 확인  
# df_클럽_팀_컬러
# df_국가_팀_컬러
# df_강화_팀_컬러
# df_관계_팀_컬러
# df_스페셜_팀_컬러

,팀 컬러,팀 컬러 설명,1단계 적용 조건,2단계 적용 조건,3단계 적용 조건,4단계 적용 조건,1단계 효과,2단계 효과,3단계 효과,4단계 효과
0,19 UEFA Champions League,19 UEFA Champions League 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
1,20 K League Best,20 K League Best 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
2,20 UEFA Champions League,20 UEFA Champions League 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
3,21 K League Best,21 K League Best 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
4,21 UEFA Champions League,21 UEFA Champions League 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
5,Ones To Watch,Ones To Watch 클래스 선수들로 구성된 팀컬러입니다.,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / / /,-
6,TOTS,역대 TOTS에 선정된 선수들로 구성된 팀컬러입니다,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / 민첩성 +2 / 스태미너 +1 /,-
7,TOTY,역대 TOTY에 선정된 선수들로 구성된 팀컬러입니다,3명,8명,10명,-,전체 능력치 +1 / / /,전체 능력치 +2 / / /,전체 능력치 +3 / 스태미너 +2 / 적극성 +1 /,-


In [60]:
## csv파일로 저장
# df_클럽_팀_컬러.to_csv('클럽팀컬러.csv', encoding='utf-8-sig', index = False)
# df_국가_팀_컬러.to_csv('국가팀컬러.csv', encoding='utf-8-sig', index = False)
# df_강화_팀_컬러.to_csv('강화팀컬러.csv', encoding='utf-8-sig', index = False)
# df_관계_팀_컬러.to_csv('관계팀컬러.csv', encoding='utf-8-sig', index = False)
# df_스페셜_팀_컬러.to_csv('스페셜팀컬러.csv', encoding='utf-8-sig', index = False)


### 2-1-3. 관계 팀 컬러 - 선수 목록 part

In [42]:
# chrome 창 열기
driver, A = open_chrome()


## 관계 팀 컬러 선수 시즌 & 이름 크롤링
# 팀컬러 기준을 만족하는 선수 리스트
TC_relation_plus = []

# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor?strTeamColorCategory=&strTeamColorType=&strTeamColorName='
driver.get(url)

# # 팝업창 닫기
# driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
# time.sleep(A)

for x in tqdm(teamcolor_relation):
   
   try:
      # 팀 컬러 타입 선택
      driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[5]/label').click() 
      time.sleep(1)
      

      # 팀 컬러 입력 & 검색
      driver.find_element_by_class_name('ui-autocomplete-input').click()
      driver.find_element_by_class_name('ui-autocomplete-input').send_keys(x)
      driver.find_element_by_class_name('btn_search').click()
      time.sleep(A)


      # 세부 설명 들어가기
      driver.find_element_by_class_name('name').click()
      driver.find_element_by_class_name('btn_detail_link').click()
      time.sleep(A) 
      
      
      # 조건 만족하는 선수 숫자 세기
      player_list = driver.find_elements_by_xpath('//*[@id="ulPlayerList"]')
      player_list_num = int(player_list[0].text.count('BP'))

      # 팀 컬러 하나의 선수 목록을 저장할 리스트
      temp_TC_relation_plus =[]
      
      # 시즌 & 선수명 추출
      for num in range(1,player_list_num+1):         
         season_temp = driver.find_element_by_xpath(f'//*[@id="ulPlayerList"]/li[{num}]/div[1]/div[3]/div[1]/img')
         player_temp = driver.find_element_by_xpath(f'//*[@id="ulPlayerList"]/li[{num}]/div[1]/div[3]/div[2]')
         season = season_temp.get_attribute('src').split('/')[-1].replace('.png','')
         player = player_temp.text

         # 시즌 & 선수명 합치기
         total_temp = season+' '+player
         temp_TC_relation_plus.append(total_temp)
         
      TC_relation_plus.append(temp_TC_relation_plus)
      
      # 세부 설명 나오기
      driver.find_element_by_class_name('btn_close').click()
      time.sleep(1)

      # 검색어 초기화 
      driver.find_element_by_class_name('btn_reset').click()
      time.sleep(1)
   
   except:
      print('error: 팀 컬러 선택 & 상세 정보 클릭')
      break
      
             
# dict로 변형 & 데이터 프레임화
dict_temp = {  '팀 컬러': [val for val in teamcolor_relation],
               '시즌 & 이름': [val for val in TC_relation_plus],             
}
columns = ['팀 컬러','시즌 & 이름']


df_TC_relation_plus = pd.DataFrame(data = dict_temp, columns=columns)
   


  0%|          | 0/206 [00:00<?, ?it/s]

In [43]:
# 결과 확인
df_TC_relation_plus

,팀 컬러,시즌 & 이름
0,01-03 성남FC 리그3연패 베스트,"[TKL 김도훈, TKL 김상식, TKL 윤정환, TKL 김영철, TKL 이기형, ..."
1,02 수원삼성 더블,"[BWC 이운재, CAP 이운재, TKL 김두현, TKL 이운재, TKL 서정원, ..."
2,09 포항 스틸러스 더블,"[TKL 최효진, 20KLB 김광석, TKL 김기동, TKL 김광석, TKL 황지수..."
3,10 FC서울 더블,"[TKL 데얀, 21KLB 김태환, TKL 정조국, TKL 최효진, TKL 고요한,..."
4,13 포항 스틸러스 더블,"[21KLB 신진호, 20KLB 김광석, TKL 김광석, TKL 김승대, TKL 황..."
...,...,...
201,푸거 코리안 듀오,"[2012KH 구자철, 2012KH 지동원, TKL 지동원, LH 지동원, 2019..."
202,프랑스 1기 황금세대,"[ICON 지네딘 지단, BWC 지네딘 지단, RMCF 지네딘 지단, ICON 티에..."
203,프랑스 2기 황금세대,"[22TOTY 킬리안 음바페, 22TOTS 킬리안 음바페, BWC 앙투안 그리즈만,..."
204,프랑스 듀오,"[ICON 파트리크 비에이라, BWC 파트리크 비에이라, ICON 클로드 마켈렐레,..."


In [45]:
# csv 파일 불러오기
df_관계_팀_컬러 = pd.read_csv('./data/team_color_crawling/관계팀컬러.csv')

# 관계 팀 컬러 적용 선수 목록 추가
df_관계_팀_컬러_plus = pd.merge(df_관계_팀_컬러, df_TC_relation_plus, on = '팀 컬러', how = 'left')
df_관계_팀_컬러_plus

# # csv 파일로 저장
# df_관계_팀_컬러_plus.to_csv('관계팀컬러.csv', encoding='utf-8-sig', index = False)

## 2-2. 선수 데이터 크롤링 (시즌 & 클럽 경력 추가 part)


각 선수별 시즌, 클럽 경력을 가져온 뒤, 'name'과 세부 능력치 기준으로 merge 예정!

In [90]:
# 선수명 csv파일 불러오기
player_name = pd.read_csv('./data/player.csv')
search_name_list = list(player_name['play_name'])

In [91]:
# 클럽 활동 연도 & 클럽 팀 합치기
def year_and_team_merge(temp_club_career):
    year = []
    team = []
    temp_club_career_2 = []

    for i in range(1,len(temp_club_career[0].split('\n'))):
        if i % 2 == 0:
            year.append(temp_club_career[0].split('\n')[i])
        else:
            team.append(temp_club_career[0].split('\n')[i])


    for i in range(len(year)):
        temp_club_career_2.append(year[i]+' / '+team[i])
        
    return temp_club_career_2

In [93]:
# chrome 창 열기
driver, A = open_chrome()

## 선수 정보 >> 각 선수의 시즌 & 클럽 경력 크롤링
# 팀컬러 기준을 만족하는 선수 리스트
name = []
season = []
club_career = []
pay_side = []
overall = []
Sprint_speed = []
Acceleration =[]
Strength = []
Stamina = []


# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter'
driver.get(url)

# # 팝업창 닫기
# driver.find_element_by_xpath('//*[@id="wrapper"]/div[1]/a/span').click()
# time.sleep(A)

for search_name in tqdm(search_name_list[:2]):
   try:
      # 선수명 입력 & 검색
      driver.find_element_by_class_name('ui-autocomplete-input').click()
      driver.find_element_by_class_name('ui-autocomplete-input').send_keys(search_name)
      driver.find_element_by_class_name('btn_search').click()
      time.sleep(A)


      # 검색된 선수 숫자 세기
      player_list = driver.find_elements_by_xpath('//*[@id="divPlayerList"]')
      player_list_num = player_list[0].text.count(f'{search_name}')
      
   
      # 시즌, 이름, 급여, 오버롤, 속력, 가속력, 몸싸움, 스태미너 추출
      for num in range(1,player_list_num+1):
         # 이름 저장
         temp_name = search_name
         name.append(temp_name)
         
         # 시즌 저장
         temp_season = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[1]/div/div[4]/div[1]/img')
         temp_season = temp_season.get_attribute('src').split('/')[-1].replace('.png','')
         season.append(temp_season)
   
         # 급여 저장
         temp_pay_side = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[2]/span').text
         pay_side.append(temp_pay_side)
       
         # 오버롤 저장
         temp_overall = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[3]/span/span').text
         overall.append(temp_overall)
         
         # 속력 저장
         temp_Sprint_speed = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[4]/span/span').text
         Sprint_speed.append(temp_Sprint_speed)
         
         # 가속력 저장
         temp_Acceleration = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[5]/span/span').text
         Acceleration.append(temp_Acceleration)
         
         # 몸싸움 저장
         temp_Strength = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[6]/span/span').text
         Strength.append(temp_Strength)
         
         # 스태미너 저장
         temp_Stamina = driver.find_element_by_xpath(f'//*[@id="divPlayerList"]/div[{num}]/div[7]/span/span').text
         Stamina.append(temp_Stamina)
         
         time.sleep(1)
      
      # 세부 설명 들어가기
      driver.find_element_by_class_name('name').click()
      driver.find_element_by_class_name('btn_detail_link').click()
      time.sleep(A)
      
      
      # 창 변경
      driver.switch_to.window(driver.window_handles[1])
      time.sleep(A)
      
      
      # 클럽 경력 추출
      temp_club_career = driver.find_element_by_xpath('//*[@id="middle"]/div/div/div[4]/div[1]/div[2]/div[2]').text
      temp_club_career_2 = year_and_team_merge(temp_club_career)
      club_career.append(temp_club_career_2)
      
      
      # 세부 설명 나오기
      driver.close()
      time.sleep(1)
      
      # 창 변경
      driver.switch_to.window(driver.window_handles[0])
      time.sleep(A)

      # 검색어 초기화 
      driver.find_element_by_class_name('btn_reset').click()
      time.sleep(1)
   
   except:
      print('error: 상세 정보 클릭')
      break
      
             
# dict로 변형 & 데이터 프레임화
dict_temp = {  'name': [val for val in name],
               'pay_side': [val for val in pay_side],
               'overall': [val for val in overall],
               'Sprint_speed': [val for val in Sprint_speed],
               'Acceleration': [val for val in Acceleration],
               'Strength': [val for val in Strength],
               'Stamina': [val for val in Stamina],
               'season': [val for val in season],
               'club_career': [val for val in club_career]                            
}
columns = ['name','pay_side','overall','Sprint_speed','Acceleration','Strength','Stamina','season','club_career']


player_info_plus = pd.DataFrame(data = dict_temp, columns=columns)

  0%|          | 0/2 [00:00<?, ?it/s]

error: 상세 정보 클릭


ValueError: All arrays must be of the same length

In [85]:
dict_temp

{'name': ['데이비드 시먼',
  '데이비드 시먼',
  '데이비드 시먼',
  '데이비드 시먼',
  '데이비드 시먼',
  '데이비드 시먼',
  '데이비드 시먼',
  '앨런 시어러',
  '앨런 시어러',
  '앨런 시어러',
  '앨런 시어러',
  '앨런 시어러',
  '앨런 시어러',
  '앨런 시어러',
  '앨런 시어러'],
 'pay_side': ['20',
  '19',
  '19',
  '16',
  '15',
  '15',
  '13',
  '25',
  '25',
  '24',
  '21',
  '21',
  '18',
  '17',
  '17'],
 'overall': ['104',
  '102',
  '101',
  '95',
  '92',
  '86',
  '86',
  '110',
  '108',
  '105',
  '100',
  '98',
  '94',
  '91',
  '87'],
 'Sprint_speed': ['57',
  '51',
  '54',
  '57',
  '57',
  '55',
  '58',
  '105',
  '103',
  '101',
  '92',
  '96',
  '90',
  '91',
  '80'],
 'Acceleration': ['73',
  '71',
  '70',
  '59',
  '55',
  '51',
  '54',
  '106',
  '103',
  '102',
  '99',
  '95',
  '92',
  '91',
  '82'],
 'Strength': ['85',
  '81',
  '86',
  '83',
  '85',
  '81',
  '84',
  '107',
  '106',
  '104',
  '97',
  '96',
  '93',
  '90',
  '88'],
 'Stamina': ['73',
  '69',
  '68',
  '64',
  '36',
  '62',
  '65',
  '108',
  '106',
  '103',
  '96',
  '95',
  '92'